In [1]:
# Tensorflow and Keras should be at their latest version for good accuracy.
# Versions used here are : 
#               tensorflow Version - 2.2.0
#               keras Version - 2.4.3
# If the system is GPU enabled, CUDA and CUDnn should be installed and added to path.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd drive/My\ Drive

/content/drive/My Drive


In [2]:
ls

001.jpeg        final_orientation_colab.ipynb    orientation1_1.h5
002.jpeg        final_orientation.ipynb          orientation1_1.py
003.jpeg        final_orientation_model3.h5      orientation1.py
004.jpeg        final_orientation_model.h5       orientation2.py
005.jpeg        generate_augmented_images.ipynb  random_lcam/
aug_0_8637.png  images/                          README.md
check2.py       mobile_0.jpeg                    train_orientation1.ipynb
check.py        modified_camera_image.py         video_to_frames.py
cuda/           modified_camera_video.py


In [14]:
# All necessary imports
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam, RMSprop
import glob
import numpy as np

In [15]:
# Check versions
import keras
import tensorflow 
print(keras.__version__)
print(tensorflow.__version__)

2.4.3
2.3.0


In [16]:
#Here Image dimensions are in the ratio 3:2. 
#It can be changed based on object shapes.

img_width = 500
img_height = 300

In [24]:
train_data_dir = 'images/train' #train images path
val_data_dir = 'images/validation'
test_data_dir = 'images/test' #test images path
epochs = 40
batch_size = 4 

In [18]:
input_shape = (img_width, img_height, 3)

In [20]:
# Training Network (Model)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [21]:
# Hyperparameters
sgd = RMSprop(lr=0.00001)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [22]:
# Print Summary of Model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 498, 298, 32)      896       
_________________________________________________________________
activation_5 (Activation)    (None, 498, 298, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 249, 149, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 247, 147, 32)      9248      
_________________________________________________________________
activation_6 (Activation)    (None, 247, 147, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 123, 73, 32)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 121, 71, 64)      

In [23]:
# Using ImageDataGenerator to use "flow_from_directory" function which automatically..
#..labels the binary classification images.
# All identical class images should be kept in seperate folders under one train folder.
# Thus for binary classification train folder will have two sub folders.
# No need to rescale the pixel values of images

train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),   
    batch_size=batch_size,
    class_mode='binary') #corrected height and width

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary') #corrected height and width

Found 90 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [25]:
#Training the model
#model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=8)
model.fit( train_generator, validation_data=val_generator, epochs=epochs)

Epoch 1/40
23/23 [==============================] - 34s 1s/step - accuracy: 0.3889 - loss: 22.6893 - val_accuracy: 0.5000 - val_loss: 6.4093
Epoch 2/40
23/23 [==============================] - 23s 999ms/step - accuracy: 0.4667 - loss: 13.4180 - val_accuracy: 0.5000 - val_loss: 5.5724
Epoch 3/40
23/23 [==============================] - 24s 1s/step - accuracy: 0.4889 - loss: 7.8614 - val_accuracy: 0.5000 - val_loss: 0.8542
Epoch 4/40
23/23 [==============================] - 25s 1s/step - accuracy: 0.5222 - loss: 4.2686 - val_accuracy: 0.5000 - val_loss: 1.6077
Epoch 5/40
23/23 [==============================] - 23s 1s/step - accuracy: 0.6222 - loss: 2.7502 - val_accuracy: 0.6667 - val_loss: 0.9004
Epoch 6/40
23/23 [==============================] - 24s 1s/step - accuracy: 0.6000 - loss: 0.8159 - val_accuracy: 0.5000 - val_loss: 0.7863
Epoch 7/40
23/23 [==============================] - 23s 1s/step - accuracy: 0.5111 - loss: 0.8489 - val_accuracy: 0.5000 - val_loss: 0.6937
Epoch 8/40
23/2

In [53]:
#Saving model
model.save('final_orientation_model3.h5')

In [61]:
# Predict the output for all test images
test_names = sorted(glob.glob("images/test/test_folder/*.jpeg"))
for name in test_names :
  img = load_img(name,target_size=(img_height,img_width)) ##corrected 
  input_arr = img_to_array(img)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  print(name,model.predict(input_arr))

images/test/test_folder/image1.jpeg [[0.03817537]]
images/test/test_folder/image15.jpeg [[1.]]
images/test/test_folder/image16.jpeg [[1.]]
images/test/test_folder/image2.jpeg [[0.02437707]]
images/test/test_folder/image3.jpeg [[0.01478617]]
images/test/test_folder/image4.jpeg [[0.01631394]]
images/test/test_folder/image5.jpeg [[0.9999999]]
images/test/test_folder/image6.jpeg [[1.]]


In [ ]:
# ImageDataGenerator function should not be used for predicting output for test images as it may give...
# ... wrong results. Reason for this can be found out.